In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dataOriDf = pd.read_excel('../../data/데이터정리3.xlsx')

In [36]:
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

#1977년 합격자부터 2019년 합격자
dataDf = dataOriDf[['년도','의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]
dataDf = dataDf[27:70]

npData = np.around(np.array(dataDf)[:,1:7])
npData

array([[ 932.,  155.,    0.,    0.,   94.,   17.],
       [ 963.,  154.,    0.,    0.,   70.,   12.],
       [ 953.,  156.,    0.,    0.,   95.,   15.],
       [1030.,  178.,    0.,    0.,   98.,   16.],
       [1026.,  132.,    0.,    0.,  149.,   26.],
       [1220.,  111.,    0.,    0.,   59.,    8.],
       [1187.,  150.,    0.,    0.,   61.,    6.],
       [1032.,  188.,    0.,    0.,  294.,   37.],
       [1242.,  181.,    0.,    0.,  139.,   25.],
       [1656.,  302.,    0.,    0.,   94.,   14.],
       [2241.,  304.,    0.,    0.,   46.,    8.],
       [2157.,  391.,    0.,    0.,  133.,   18.],
       [2194.,  526.,    0.,    0.,  180.,   33.],
       [1983.,  493.,    0.,    0.,  260.,   62.],
       [2195.,  535.,    0.,    0.,  187.,   47.],
       [2116.,  550.,    0.,    0.,  199.,   48.],
       [2400.,  616.,    0.,    0.,   75.,   20.],
       [2282.,  522.,    0.,    0.,   81.,   21.],
       [1602.,  354.,    0.,    0.,  692.,  158.],
       [1321.,  337.,    0.,   

# 시그모이드 변형 모델

In [33]:
# 분류모델함수 생성
# x - x값, sp - 시작점, xl - x축 조정, yl - y축 조정
def sigmoid_function(x, sp, xl, yl):
    
    result = 1 / ( yl+np.exp((x-(sp+(10/xl/2)))*xl) )
    
    for i in range(np.size(x)):
        if x[i]<sp:
           result[i]=0
        if x[i]>(sp+(10/xl)):
           result[i]=0
            
    return result

# 분류모델 확률질량함수 생성
def makeClusterModel(sp, xl, yl):
    x = np.arange(1, 101, 1)
    y = sigmoid_function(x, sp, xl, yl)
    sum = np.sum(y)
    z = y/sum
    
    return z

# 사망률 적용

In [34]:
def deathRateArray(array):
    deathWoman = pd.read_excel('../../data/사망확률여자.xlsx')[7:,:101].iloc.set_index(['연도'])
    deathMan = pd.read_excel('../../data/사망확률남자.xlsx')[7:,:101].iloc.set_index(['연도'])
    deathRate = np.array([deathWoman, deathMan])
    resultData = np.around(deathRate*array)
    
    return resultData

# 시그모이드 

In [35]:
def makeArrayUseModel(tuningList):
    model1 = makeClusterModel(26,tuningList[0][0],tuningList[1][0]) # 의대 남
    model2 = makeClusterModel(26,tuningList[0][1],tuningList[1][1]) # 의대 여
    model3 = makeClusterModel(28,tuningList[0][2],tuningList[1][2]) # 의전원 남
    model4 = makeClusterModel(28,tuningList[0][3],tuningList[1][3]) # 의전원 여
    model5 = makeClusterModel(27,tuningList[0][4],tuningList[1][4]) # 재시험 남
    model6 = makeClusterModel(27,tuningList[0][5],tuningList[1][5]) # 재시험 여
    result = [model1, model2, model3, model4, model5, model6]
    
    return np.array(result)

# 신규인원 생성

In [52]:
def makeNewPerson(npData, tuningList,):
    oldSize = 100
    yearSize = len(npData)
    modelSize = 6
    
    #모델 적용 배열
    applyModelArray = np.zeros((yearSize,modelSize,oldSize))

    #신규인원 배열
    newPersonArray = np.zeros((yearSize,2,oldSize))

    
    modelAry = makeArrayUseModel(xlList, ylList)
    for i in range(yearSize):
        for j in range(modelSize):
            applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))

            if j%2==0:
                newPersonArray[i][0] += applyModelArray[i][j]
            else:
                newPersonArray[i][1] += applyModelArray[i][j]
    return newPersonArray

# 최종인원 도출

In [ ]:
def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    return dataArray

def makeResultPersonArray(newPersonArray, deathRateArray, retireeRateArray):
    #누적인원 배열
    
    resultPersonArray = np.zeros((yearSize,2,oldSize))

    for i in range(yearSize):
        if i>0:
            resultPersonArray[i] = shiftOld(resultPersonArray[i-1]) + newPersonArray[i]
        else:
            resultPersonArray[i] =  newPersonArray[i]
    return resultPersonArray

In [ ]:
tuningList = [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],[0.005, 0.005, 0.005, 0.005, 0.005, 0.005]]
modelSize = len(makeArrayUseModel(tuningList))

makeNewPerson

makeNewPerson()


